In [ ]:

from deephive.environment.utils import parse_config
from datetime import datetime
import os 
import numpy as np
import neptune
import torch
from plot_utils import *
from deephive.environment.optimization_environment import OptimizationEnv
from deephive.environment.optimization_functions.benchmark_functions import FunctionSelector
from deephive.environment.utils import mean_confidence_interval
import pandas as pd
function_selector = FunctionSelector()
from deephive.policies.mappo import MAPPO
np.set_printoptions(suppress=True, precision=4)
from dotenv import load_dotenv
api_token = os.environ.get("NEPTUNE_API_TOKEN")
load_dotenv()

## DEEPHIVE TRAINING

In [ ]:
config_path = "../config/exp_config.json"
model_path = "../models/pbest_unfreeze.pth"
model_path_2 = "../models/gbest.pth"
config = parse_config(config_path)
config['use_gbest'] = False
config['use_lbest'] = False
config["use_optimal_value"] = True
config["log_scale"] = True
config["include_gbest"] = False
config["negative"] = True
if config["include_gbest"] or config["use_lbest"]:
    config["obs_dim"] = 5
else:
    config["obs_dim"] = 4
config["ep_length"] = 25

config["min_action_std"] = 0.001
config["action_std"] = 0.2
config["variable_std"] = False
config["update_timestep"] = 2
config["decay_rate"] = 0.99
config["log_interval"] = 500
config["decay_interval"] = 5
config["save_interval"] = 200
config["test_decay_rate"] = 0.9
config["test_decay_start"] = 0
config["reward_scheme"] = "FullRewardScheme2"
config["observation_scheme"]= "SimpleObservationScheme"
config["n_episodes"] = 5000
config["plot_gif"] = True
config["plot_gbest"] = True
config["test_ep_length"] = 100
config["n_agents"] = 12
config["n_dim"] = 2
config['objective_function'] = "BenchmarkFunctions" 
config["function_id"] = "f01"
config["neighborhood_size"] = 4
config["topology"] = "random"

mode = "train"

def initialize(config, mode="train", **kwargs):
    env = OptimizationEnv(config)
    agent_policy = MAPPO(config)
    if mode == "test" or mode == "benchmark":
        model_path = kwargs.get("model_path", None)
        if model_path is None:
            raise ValueError("Model path must be provided for testing")
        # check if model path is a list of paths
        if isinstance(model_path, list):
            agent_policies = []
            for path in model_path:
                agent_policy = MAPPO(config)
                agent_policy.load(path)
                agent_policies.append(agent_policy)
            return env, agent_policies
        else:
            agent_policy.load(model_path)
            return env, agent_policy
    else:
        return env, agent_policy
    

def get_action(observation, agent_policy, env, observation_std=None, **kwargs):
    # Ensure observation_info is a numpy array
    
    if not isinstance(observation, np.ndarray):
        observation = np.array(observation)
        assert observation.shape[0] == env.n_dim, "Observation must have the same number of dimensions as the environment"

    # Initialize observation_std with zeros or use provided std, ensuring it matches the shape of observation
    if observation_std is None:
        observation_std = np.zeros_like(observation)
    else:
        observation_std = np.array(observation_std)

    # Flatten the observation and std arrays
    observation_flat = observation.reshape(env.n_dim * env.n_agents, -1)  # Flatten to 1D array
    observation_std_flat = observation_std.reshape(-1)  # Flatten to 1D array
    # Pass the entire flattened observation and std arrays to select_action
    action_flat = agent_policy.select_action(observation_flat, observation_std_flat)

    # Reshape the flattened action array back to the original (n_agents, n_dim) shape
    actions = action_flat.reshape(env.n_dim, env.n_agents).T  # Reshape to (n_agents, n_dim

    return actions  # Return the action


def get_direct_action(obs, obs_std, agent_policy):
    torch_obs = torch.FloatTensor(obs)
    torch_obs_std = torch.FloatTensor(obs_std)
    action = agent_policy.policy.act(torch_obs, torch_obs_std)
    return action

In [ ]:
env, agent_policy = initialize(config, mode="train", model_path=None)
obs = env.reset()

In [ ]:
def initialize_logger(api_token, tags, config, mode="train"):
        run = neptune.init_run(
        project="DMO-LAB/DeepHive-V2",
        # source files = all python files in the current directory,
        source_files=["*.py"],
        api_token=api_token,
        tags=[tags, mode, config["objective_function"], str(config["layer_size"])]
        )
        return run

def train(env, agent_policy, config, title="experiment_1", **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
        
    save_path = kwargs.get("save_path", "training_results/")
    save_path = os.path.join(save_path, title)
    # make directory if it does not exist
    os.makedirs(save_path, exist_ok=True)
    n_episodes = config["n_episodes"]
    average_returns = []  
    timestep = 0    
    for i in range(n_episodes):
        #print(f"Episode {i} started, timestep {timestep}")
        obs, obs_std = env.reset()
        episode_return = np.zeros(env.n_agents)
        for step in range(env.ep_length):
            #print(f"Episode {i}, step {step}, timestep {timestep}")
            actions = get_action(obs, agent_policy, env, obs_std)
            obs, reward, done, info = env.step(actions)
            for ag in range(env.n_agents):
                agent_policy.buffer.rewards += [reward[ag]] * env.n_dim
                agent_policy.buffer.is_terminals += [done[ag]] * env.n_dim
            episode_return += reward
            obs, obs_std = obs
            timestep += 1
            if step == env.ep_length - 1:
                average_returns.append(np.mean(episode_return))
                if neptune_logger is not None:
                    neptune_logger["train/average_return"].log(average_returns[-1])
                running_average_rewards = np.mean(average_returns)
                
            if neptune_logger and i % config["log_interval"] == 0:
                neptune_logger[f"Episode_{i}/gbest_values"].log(env.gbest[-1])

        if i % config["update_timestep"] == 0 and timestep > 0:
            #print(f"Updating policy at episode {i}")
            agent_policy.update()
        if i % config["log_interval"] == 0 and timestep > 0:
            print(f"Episode {i} completed")
            print(f"Average return: {running_average_rewards}")
            if env.n_dim == 2:
                env.render(type="history", file_path=f"{save_path}/episode_{i}.gif")  
                if neptune_logger:
                    neptune_logger[f"train/gifs/{i}.gif"].upload(f"{save_path}/episode_{i}.gif")
        if i % config["decay_interval"] == 0 and timestep > 0:
            agent_policy.decay_action_std(config["decay_rate"], min_action_std=config["min_action_std"], debug=False)
        if i % config["save_interval"] == 0 and timestep > 0:
            if average_returns[-1] > running_average_rewards:
                print(f"Saving model at episode {i} with average return {average_returns[-1]} and running average {running_average_rewards}")
                agent_policy.save(save_path, episode=i)
        
    return agent_policy

In [ ]:
# title = "experiment_104"
# tags = "training with 4 observation using pbest and log scale"
# neptune_logger = initialize_logger(api_token, title, config, mode="train")
# agent_policy = train(env, agent_policy, config, title=title, neptune_logger=neptune_logger)
# neptune_logger.stop()

## DEEPHIVE TEST

In [ ]:
def test(env, agent_policy, iters, decay_start=100, decay_rate=0.995, 
         min_action_std=0.001, max_action_std=0.5, 
         save_gif = False, save_path = "test_results", function_id=0, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    os.makedirs(save_path, exist_ok=True)
    all_gbest_vals = []
    print(f"Testing function {function_id} with {env.n_agents} agents and {env.n_dim} dimensions")
    for iter in range(iters):
        gbest_vals = []
        obs = env.reset()[0]
        agent_policy.set_action_std(max_action_std)
        current_action_std = agent_policy.action_std
        for step in range(env.ep_length):
            action = get_action(obs, agent_policy, env)
            obs, _ , _ , _ = env.step(action)
            obs = obs[0]
            gbest_vals.append(env.gbest[-1])
            if neptune_logger:
                neptune_logger[f"test/{function_id}/Episode_{iter}/gbest_values"].log(env.gbest[-1])
            if step >= decay_start:
                # Decay the std uniformly from the max to the min std over the specified rate
                current_action_std = max(min_action_std, current_action_std * decay_rate)
                agent_policy.set_action_std(current_action_std)   
        if env.n_dim == 2 and save_gif:
            env.render(type="history", file_path=f"{save_path}/episode_{iter}.gif") 
            if neptune_logger:
                neptune_logger[f"test/{function_id}/gifs/{iter}.gif"].upload(f"{save_path}/episode_{iter}.gif")
        all_gbest_vals.append(np.array(gbest_vals))
        print(f"Final gbest value: {env.gbest[-1]} at iteration {iter}")
    
    np.save(f"{save_path}/{function_id}_gbest_history.npy", np.array(all_gbest_vals))
    return all_gbest_vals

def analyze_results(base_path, model_lists, model_path_list, successful_functions, function_selector, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    # Define a multi-level column structure: each optimizer has the same sub-columns
    columns = pd.MultiIndex.from_product([model_lists, 
                                          ["mean", "lower", "upper", "optimum", "error"]],
                                         names=['optimizer', 'metric'])
    # Initialize an empty DataFrame with these columns
    df = pd.DataFrame(columns=columns)
    
    for function_id in successful_functions:
        row_data = {}
        function_info = function_selector.get_function(function_id)
        function_opt_val = function_info["global_min"]
        
        # Loop through each optimizer
        for optimizer, model_path in zip(model_lists, model_path_list):
            data_path = model_path + f"deephive/{function_id}/{function_id}_gbest_history.npy"
            try:
                # Attempt to load the optimizer's result and compute metrics
                gbest_values = np.load(data_path) * -1
                mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
                error_val = abs(mean_val[-1] - function_opt_val)
                row_data[(optimizer, 'mean')] = mean_val[-1]
                row_data[(optimizer, 'lower')] = lower_val[-1]
                row_data[(optimizer, 'upper')] = upper_val[-1]
                row_data[(optimizer, 'optimum')] = function_opt_val
                row_data[(optimizer, 'error')] = error_val
                
                # Additional logging or plotting can be added here as needed
                
            except Exception as e:
                print(f"Error processing {optimizer} for function {function_id}: {e}")
                # Fill missing values if any error occurs
                for metric in ["mean", "lower", "upper", "optimum", "error"]:
                    row_data[(optimizer, metric)] = np.nan
        
        # After collecting data for all optimizers, add the row to the DataFrame
        #df = df.append(pd.Series(row_data, name=function_id))
        df.loc[function_id] = row_data
    
    # Save the DataFrame to CSV
    df.to_csv(f"{base_path}/results.csv")
    
    # Log the DataFrame if Neptune logger is provided
    if neptune_logger:
        neptune_logger["test/results"].upload(f"{base_path}/results.csv")
        
    return df

def run_test(function_ids, iters, save_dir, model_path, config, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    successful_functions = []
    for function_id in function_ids:
        try:
            config["function_id"] = function_id
            function_dim = function_selector.get_function(function_id)["dimension"]
            if config["n_dim"] > function_dim:
                print(f"function {function_id} has {function_dim} dimensions, setting n_dim to {function_dim}")
                config["n_dim"] = function_dim
            env, agent_policy = initialize(config, mode="test", model_path=model_path)
            _ = env.reset()[0]
            agent_policy.load(model_path)
            save_path = f"{save_dir}/{function_id}"
            try:
                all_gbest_vals = test(env, agent_policy, iters, decay_start=0, decay_rate=0.9, 
                                min_action_std=0.0001, max_action_std=0.5, 
                                save_gif = False, save_path = save_path, function_id=function_id, neptune_logger=neptune_logger)
                successful_functions.append(function_id)
            except Exception as e:
                print(f"Function {function_id} failed with error {e}")
        except Exception as e:
            print(f"Function {function_id} failed with error {e}")
        
    #df = analyze_results(save_dir, successful_functions, function_selector, neptune_logger=neptune_logger)
    if neptune_logger: 
        neptune_logger.stop()
    return successful_functions, save_dir, env, agent_policy


def run_test_deephive(function_ids, iters, save_dir, model_path, config, **kwargs):
    dimension = config["n_dim"]
    neptune_logger = kwargs.get("neptune_logger", None)
    successful_functions = []
    for function_id in function_ids:
        try:
            config["function_id"] = function_id
            function_dim = function_selector.get_function(function_id)["dimension"]
            if config["n_dim"] > function_dim:
                print(f"function {function_id} has {function_dim} dimensions, setting n_dim to {function_dim}")
                config["n_dim"] = function_dim
            else:
                config["n_dim"] = dimension
            
            
            env, agent_policy = initialize(config, mode="test", model_path=model_path)
            _ = env.reset()[0]
            agent_policy.load(model_path)
            
            try:
                test_save_path = f"{save_dir}/deephive/{function_id}"
                all_gbest_vals = test(env, agent_policy, iters, decay_start=10, decay_rate=0.9, 
                                min_action_std=0.0001, max_action_std=0.5, 
                                save_gif = False, save_path = test_save_path, function_id=function_id, neptune_logger=neptune_logger)
                plot_individual_function_evaluation(all_gbest_vals, config["n_agents"], f"{test_save_path}/{function_id}_individual.png", log_scale=config["log_scale"])
                num_function_evaluation(all_gbest_vals, config["n_agents"], f"{test_save_path}/{function_id}_num_evaluations.png", log_scale=config["log_scale"])
                if neptune_logger:
                    neptune_logger[f"deephive/{function_id}/individual"].upload(f"{test_save_path}/{function_id}_individual.png")
                    neptune_logger[f"deephive/{function_id}/num_evaluations"].upload(f"{test_save_path}/{function_id}_num_evaluations.png")
            except Exception as e:
                print(f"Function {function_id} failed with error {e}")
            successful_functions.append(function_id)
        except Exception as e:
            print(f"Function {function_id} failed with error {e}")
        
    #df = analyze_results(save_dir, successful_functions, function_selector, neptune_logger=neptune_logger)
    if neptune_logger: 
        neptune_logger.stop()
    return successful_functions, save_dir



In [ ]:
function_ids = ["f01"]#, "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19"]
config["n_agents"] = 40
config["n_dim"] = 30
config['objective_function'] = "BenchmarkFunctions"
config["ep_length"] = 100
config["log_scale"] = True
config["use_lbest"] = True
config["obs_dim"] = 5
MODEL_PATH = "training_results/experiment_103/policy-4800.pth"
config["neighborhood_size"] = 10
env, agent_policy = initialize(config, mode="test", model_path=MODEL_PATH)
iters = 1
save_dir = f"new_test_results/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/" 
title = "experiment_103"
tags = "testing with 40 agents and log scale"
# neptune_logger = None #initialize_logger(api_token, title, config, mode="test")
# successfull_functions, save_dir, env, policy = run_test(function_ids, iters, save_dir, MODEL_PATH, config, neptune_logger=neptune_logger)
_ = env.reset()[0]
env.compute_lbest()
len(set(env.lbest_positions[:, 0].tolist()))

In [ ]:
model_experiments = [102, 103, 104]
model_lists = ["gbest", "lbest", "pbest"]
obs_dim = [4, 5, 4]
function_ids = ["f01", "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
                "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29", "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "f38", "f39", "f40",
                "f41", "f42", "f43", "f44", "f45", "f46", "f47", "f48", "f49", "f50"]
#function_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
config["n_agents"] = 40
config["n_dim"] = 30
config['objective_function'] = "BenchmarkFunctions"
config["ep_length"] = 1000
config["log_scale"] = False
iters = 10
config["neighborhood_size"] = 10

base_save_dir = f"new_test_results/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/" 
for i, model in enumerate(model_experiments):
    config["n_dim"] = 30
    config["n_agents"] = 40
    config["obs_dim"] = obs_dim[i]
    if i==1:
        config["use_lbest"] = True
    else:
        config["use_lbest"] = False
    MODEL_PATH = f"training_results/experiment_{model}/policy-4800.pth"
    # env, agent_policy = initialize(config, mode="test", model_path=MODEL_PATH)
    title = f"experiment_{model}"
    save_dir = base_save_dir + f"model{model}/"
    tags = f"testing with 40 agents and log scale"
    neptune_logger = None#initialize_logger(api_token, title, config, mode="test")
    successful_functions, save_dir = run_test_deephive(function_ids, iters, save_dir, MODEL_PATH, config, neptune_logger=neptune_logger)
    if neptune_logger:
        neptune_logger.stop()
    print(f"Experiment {model} completed")
    

In [ ]:

save_dir =base_save_dir #f"new_test_results/2024-03-06_22-07-01/"
model_paths = [save_dir + f"model{model}/" for model in [102, 103, 104]]
model_lists = ["gbest", "lbest", "pbest"]
df = analyze_results(save_dir, model_lists, model_paths, successful_functions, function_selector, neptune_logger=neptune_logger)

In [ ]:
df.to_csv(f"{save_dir}/results.csv")

## PSO-BSA

In [ ]:
class PSOBSA:
    def __init__(self, fitness_function, dimension, swarm_size, inertia_weight, 
                 acc_coefficients, mix_rate, mutation_probability, neighborhood_size,
                 lower_bound=-100, upper_bound=100, scale=False, apply_mutation=False):
        self.scale_option = scale
        self.num_evaluations = 0
        self.lower_bound = lower_bound  
        self.upper_bound = upper_bound
        self.obj_function = fitness_function
        self.dimension = dimension
        self.apply_mutation = apply_mutation
        self.swarm_size = swarm_size
        self.inertia_weight = inertia_weight
        self.acc_coefficients = acc_coefficients  # Tuple (c1, c2)
        self.mix_rate = mix_rate
        self.mutation_probability = mutation_probability
        self.neighborhood_size = neighborhood_size
        self.positions = np.random.uniform(low=lower_bound, high=upper_bound, size=(swarm_size, dimension))
        if self.scale_option:
            self.positions = self.scale(self.positions)
        self.velocities = np.zeros((swarm_size, dimension))
        self.fitness_values = self.fitness_function(self.positions)
        self.pbest_positions = np.copy(self.positions)
        self.pbest_values = np.copy(self.fitness_values)
        self.gbest_position = self.pbest_positions[np.argmin(self.pbest_values)]
        self.gbest_values = self.pbest_values.min()
        self.gbest_history = []
        self.gbest_history.append(self.gbest_values)
        
    def fitness_function(self, positions, no_tracking=False):
        if not no_tracking:
            self.num_evaluations += len(positions)
        if self.scale_option:
            positions = self.unscale(positions)
        return self.obj_function(positions)
        
    def scale(self, positions):
        return 2 * (positions - self.lower_bound) / (self.upper_bound - self.lower_bound) - 1
    
    def unscale(self, positions):
        return ((positions + 1) / 2) * (self.upper_bound - self.lower_bound) + self.lower_bound
    
    def initialize_neighborhoods(self):
        self.neighborhoods = []
        for i in range(self.swarm_size):
            neighborhood_indices = list(range(i - self.neighborhood_size, i)) + list(range(i + 1, i + 1 + self.neighborhood_size))
            neighborhood_indices = [index % self.swarm_size for index in neighborhood_indices]
            self.neighborhoods.append(neighborhood_indices)
    
    def compute_lbest(self):
        self.lbest_positions = np.zeros((self.swarm_size, self.dimension))
        for i in range(self.swarm_size):
            neighborhood_fitnesses = self.fitness_values[self.neighborhoods[i]]
            best_neighbor_idx = self.neighborhoods[i][np.argmin(neighborhood_fitnesses)]
            self.lbest_positions[i] = self.positions[best_neighbor_idx]
    
    def update_velocity(self):
        r1, r2 = np.random.rand(self.swarm_size, self.dimension), np.random.rand(self.swarm_size, self.dimension)
        cognitive_component = self.acc_coefficients[0] * r1 * (self.pbest_positions - self.positions)
        social_component = self.acc_coefficients[1] * r2 * (self.gbest_position - self.positions)
        self.velocities = self.inertia_weight * self.velocities + cognitive_component + social_component
    
    def update_positions(self):
        self.positions += self.velocities
        if self.scale_option:
            self.positions = np.clip(self.positions, -1, 1)  # Scaled bounds
        else:
            self.positions = np.clip(self.positions, self.lower_bound, self.upper_bound)
    
    def calculate_fitness(self):
        self.fitness_values = self.fitness_function(self.positions)
        self.update_pbest_and_gbest()
        
    def update_pbest_and_gbest(self):
        better_mask = self.fitness_values < self.pbest_values
        self.pbest_positions[better_mask] = self.positions[better_mask]
        self.pbest_values[better_mask] = self.fitness_values[better_mask]
        gbest_candidate_idx = np.argmin(self.pbest_values)
        if self.pbest_values[gbest_candidate_idx] < self.gbest_values:
            self.gbest_position = self.pbest_positions[gbest_candidate_idx]
            self.gbest_values = self.pbest_values[gbest_candidate_idx]
            
    def mutate(self, positions, lbest_positions):
        A = 2 * np.random.rand(self.swarm_size, self.dimension)
        phi = np.random.rand(self.swarm_size, self.dimension)
        return positions + A * (phi * lbest_positions - positions)
    
    def crossover(self, positions, trial_positions):
        crossover_mask = np.random.rand(self.swarm_size, self.dimension) < self.mix_rate
        return np.where(crossover_mask, trial_positions, positions)

    def mutate_and_crossover(self):
        for i in range(self.swarm_size):
            if np.random.rand() < self.mutation_probability:
                trial_positions = self.mutate(self.positions, self.lbest_positions[i])
            else:
                self.permute_lbest(i)
                trial_positions = self.mutate(self.positions, self.lbest_positions[i])
            new_positions = self.crossover(self.positions, trial_positions)
            new_fitness = self.fitness_function(new_positions)
            better_mask = new_fitness < self.fitness_values
            self.positions[better_mask] = new_positions[better_mask]
            self.fitness_values[better_mask] = new_fitness[better_mask]
            self.update_pbest_and_gbest()
            # if np.min(new_fitness) < self.fitness_function(self.gbest_position.reshape(1, -1)):
            #     self.gbest_position = new_positions[np.argmin(new_fitness)]
            
    
    def permute_lbest(self, particle_index):
        neighbor_indices = self.neighborhoods[particle_index]
        np.random.shuffle(neighbor_indices)
        self.lbest_positions[particle_index] = self.positions[neighbor_indices[0]]
    
    def optimize(self, iterations):
        self.initialize_neighborhoods()
        for _ in range(iterations):
            self.compute_lbest()
            self.update_velocity()
            self.update_positions()
            self.calculate_fitness()
            if self.apply_mutation:
                self.mutate_and_crossover()
            self.gbest_history.append(self.gbest_values)
        return np.array(self.gbest_history)
   
function_id = "f01"
function_info = function_selector.get_function(function_id)
fitness_function = function_info["func"]
dimension = function_info["dimension"]
lower_bound = function_info["domain"][0]
upper_bound = function_info["domain"][1]
global_min = function_info["global_min"] 
swarm_size = 40
apply_mutation = True
pso_bsa = PSOBSA(fitness_function=fitness_function,
                dimension=dimension,
                swarm_size=swarm_size,
                inertia_weight=0.7,
                acc_coefficients=(1.4, 1.4),
                mix_rate=1,
                mutation_probability=0.2,
                neighborhood_size=10,
                scale=False,
                lower_bound=lower_bound,
                upper_bound=upper_bound,
                apply_mutation=apply_mutation)

gbest_history = pso_bsa.optimize(iterations=100)
print(gbest_history[-1])
print(pso_bsa.num_evaluations)

In [ ]:
def run_pso(function_id, iters, save_dir, ep_length, apply_mutation=False, swarm_size=40, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    try:
        save_dir = f"{save_dir}/{function_id}"
        os.makedirs(save_dir, exist_ok=True)
        function_info = function_selector.get_function(function_id)
        fitness_function = function_info["func"]
        dimension = kwargs.get("dimension", function_info["dimension"])
        lower_bound = function_info["domain"][0]
        upper_bound = function_info["domain"][1]
        global_min = function_info["global_min"]
        all_gbest_vals = []
        for iter in range(iters):
            pso_bsa = PSOBSA(fitness_function=fitness_function,
                dimension=dimension,
                swarm_size=swarm_size,
                inertia_weight=0.7,
                acc_coefficients=(1.4, 1.4),
                mix_rate=1,
                mutation_probability=0.2,
                neighborhood_size=10,
                scale=False,
                lower_bound=lower_bound,
                upper_bound=upper_bound,
                apply_mutation=apply_mutation)
            gbest_history = pso_bsa.optimize(iterations=ep_length)
            all_gbest_vals.append(gbest_history)
        np.save(f"{save_dir}/{function_id}_gbest_history.npy", np.array(all_gbest_vals))
        if neptune_logger:
            neptune_logger[f"pso/{function_id}/gbest_history"].upload(f"{save_dir}/{function_id}_gbest_history.npy")
    except Exception as e:
        print(f"Function {function_id} failed with error {e}")
    return np.array(all_gbest_vals)

In [ ]:
def run_test(function_ids, iters, save_dir, model_path, config, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    successfull_functions = []
    for function_id in function_ids:
        try:
            config["function_id"] = function_id
            function_dim = function_selector.get_function(function_id)["dimension"]
            if config["n_dim"] > function_dim:
                print(f"function {function_id} has {function_dim} dimensions, setting n_dim to {function_dim}")
                config["n_dim"] = function_dim
            env, agent_policy = initialize(config, mode="test", model_path=model_path)
            _ = env.reset()[0]
            agent_policy.load(model_path)
            
            try:
                test_save_path = f"{save_dir}/deephive/{function_id}"
                all_gbest_vals = test(env, agent_policy, iters, decay_start=0, decay_rate=0.9, 
                                min_action_std=0.0001, max_action_std=0.5, 
                                save_gif = False, save_path = test_save_path, function_id=function_id, neptune_logger=neptune_logger)
                plot_individual_function_evaluation(all_gbest_vals, config["n_agents"], f"{test_save_path}/{function_id}_individual.png", log_scale=config["log_scale"])
                num_function_evaluation(all_gbest_vals, config["n_agents"], f"{test_save_path}/{function_id}_num_evaluations.png", log_scale=config["log_scale"])
                if neptune_logger:
                    neptune_logger[f"deephive/{function_id}/individual"].upload(f"{test_save_path}/{function_id}_individual.png")
                    neptune_logger[f"deephive/{function_id}/num_evaluations"].upload(f"{test_save_path}/{function_id}_num_evaluations.png")
            except Exception as e:
                print(f"Function {function_id} failed with error {e}")
            try:
                pso_save_path = f"{save_dir}/pso/"
                apply_mutation = False
                pso_gbest_history = run_pso(function_id=function_id, iters=iters, save_dir=pso_save_path, 
                                            ep_length=config["ep_length"], apply_mutation=apply_mutation, 
                                            neptune_logger=neptune_logger, swarm_size=config["n_agents"], dimension=config["n_dim"])
                print(f"PSO gbest history: {pso_gbest_history.shape}")
                plot_individual_function_evaluation(pso_gbest_history, config["n_agents"], f"{pso_save_path}/{function_id}_individual.png", log_scale=config["log_scale"])
                num_function_evaluation(pso_gbest_history, config["n_agents"], f"{pso_save_path}/{function_id}_num_evaluations.png", log_scale=config["log_scale"])
                if neptune_logger:
                    neptune_logger[f"pso/{function_id}/individual"].upload(f"{pso_save_path}/{function_id}_individual.png")
                    neptune_logger[f"pso/{function_id}/num_evaluations"].upload(f"{pso_save_path}/{function_id}_num_evaluations.png")
            except Exception as e:
                import traceback; traceback.print_exc();
                print(f"Function {function_id} failed with error {e}")
            try:
                psobsa_save_path = f"{save_dir}/psobsa/"
                apply_mutation = True
                psobsa_gbest_history = run_pso(function_id=function_id, iters=iters, save_dir=psobsa_save_path, ep_length=config["ep_length"],
                                               apply_mutation=apply_mutation, neptune_logger=neptune_logger, swarm_size=config["n_agents"], dimension=config["n_dim"])
                print(f"PSO-BSA gbest history: {psobsa_gbest_history.shape}")
                plot_individual_function_evaluation(psobsa_gbest_history, config["n_agents"], f"{psobsa_save_path}/{function_id}_individual.png", log_scale=config["log_scale"])
                num_function_evaluation(psobsa_gbest_history, config["n_agents"], f"{psobsa_save_path}/{function_id}_num_evaluations.png", log_scale=config["log_scale"])
                if neptune_logger:
                    neptune_logger[f"psobsa/{function_id}/individual"].upload(f"{psobsa_save_path}/{function_id}_individual.png")
                    neptune_logger[f"psobsa/{function_id}/num_evaluations"].upload(f"{psobsa_save_path}/{function_id}_num_evaluations.png")
            except Exception as e:
                import traceback; traceback.print_exc();    
                print(f"Function {function_id} failed with error {e}")
            successfull_functions.append(function_id)
        except Exception as e:
            print(f"Function {function_id} failed with error {e}")
        
    df = analyze_results(save_dir, successfull_functions, function_selector, neptune_logger=neptune_logger)
    if neptune_logger: 
        neptune_logger.stop()
    return successfull_functions, save_dir, df


In [ ]:
function_ids = ["f01", "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19"]
config["n_agents"] = 40
config["n_dim"] = 30
config['objective_function'] = "BenchmarkFunctions" 
config["ep_length"] = 100 
config["log_scale"] = True

MODEL_PATH = "training_results/experiment_101/policy-4800.pth"
env, agent_policy = initialize(config, mode="test", model_path=MODEL_PATH)
iters = 1
save_dir = f"new_test_results/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}/" 
title = "experiment_103"
tags = "testing with 40 agents and log scale"
neptune_logger = None #initialize_logger(api_token, title, config, mode="test")
successfull_functions, save_dir, df = run_test(function_ids, iters, save_dir, MODEL_PATH, config, neptune_logger=neptune_logger)


In [ ]:
analyze_results(save_dir, successfull_functions, function_selector, neptune_logger=neptune_logger)

In [ ]:
f = np.load("/Users/elotech/Desktop/DeepHiveV2/notebooks/new_test_results/2024-03-04_17-55-16/pso/f01/f01_gbest_history.npy")

In [ ]:
f.shape

In [ ]:
# def analyze_results(base_path, successfull_functions, function_selector, **kwargs):
#     neptune_logger = kwargs.get("neptune_logger", None)
#     # Define a multi-level column structure: each optimizer has the same sub-columns
#     columns = pd.MultiIndex.from_product([['deephive', 'pso', 'psobsa'], 
#                                           ["mean", "lower", "upper", "optimum", "error"]],
#                                          names=['optimizer', 'metric'])
#     # Initialize an empty DataFrame with these columns
#     df = pd.DataFrame(columns=columns)
    
#     for function_id in successfull_functions:
#         row_data = {}
#         function_info = function_selector.get_function(function_id)
#         function_opt_val = function_info["global_min"]
        
#         # Loop through each optimizer
#         for optimizer in ['deephive', 'pso', 'psobsa']:
#             data_path = base_path + f"{optimizer}/{function_id}/{function_id}_gbest_history.npy"
#             try:
#                 # Attempt to load the optimizer's result and compute metrics
#                 gbest_values = np.load(data_path) * -1
#                 mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
#                 error_val = abs(mean_val[-1] - function_opt_val)
#                 row_data[(optimizer, 'mean')] = mean_val[-1]
#                 row_data[(optimizer, 'lower')] = lower_val[-1]
#                 row_data[(optimizer, 'upper')] = upper_val[-1]
#                 row_data[(optimizer, 'optimum')] = function_opt_val
#                 row_data[(optimizer, 'error')] = error_val
                
#                 # Additional logging or plotting can be added here as needed
                
#             except Exception as e:
#                 print(f"Error processing {optimizer} for function {function_id}: {e}")
#                 # Fill missing values if any error occurs
#                 for metric in ["mean", "lower", "upper", "optimum", "error"]:
#                     row_data[(optimizer, metric)] = np.nan
        
#         # After collecting data for all optimizers, add the row to the DataFrame
#         #df = df.append(pd.Series(row_data, name=function_id))
#         df.loc[function_id] = row_data
    
#     # Save the DataFrame to CSV
#     df.to_csv(f"{base_path}/results.csv")
    
#     # Log the DataFrame if Neptune logger is provided
#     if neptune_logger:
#         neptune_logger["test/results"].upload(f"{base_path}/results.csv")
        
#     return df

In [ ]:
# save_dir = f"new_test_results/2024-03-04_17-31-05/" 
# analyze_results(save_dir, ["f01", "f02"], function_selector, neptune_logger=neptune_logger)

## DATA ANALYSIS

In [ ]:
from deephive.environment.optimization_functions.benchmark_functions import FunctionSelector
from deephive.environment.utils import mean_confidence_interval
function_selector = FunctionSelector()
import numpy as np
import pandas as pd

In [ ]:
def analyze_results(base_path, model_lists, model_path_list, successful_functions, function_selector, **kwargs):
    neptune_logger = kwargs.get("neptune_logger", None)
    # Define a multi-level column structure: each optimizer has the same sub-columns
    columns = pd.MultiIndex.from_product([model_lists, 
                                          ["mean", "lower", "upper", "optimum", "error"]],
                                         names=['optimizer', 'metric'])
    # Initialize an empty DataFrame with these columns
    df = pd.DataFrame(columns=columns)
    
    for function_id in successful_functions:
        row_data = {}
        function_info = function_selector.get_function(function_id)
        function_opt_val = function_info["global_min"]
        
        # Loop through each optimizer
        for optimizer, model_path in zip(model_lists, model_path_list):
            data_path = model_path + f"deephive/{function_id}/{function_id}_gbest_history.npy"
            try:
                # Attempt to load the optimizer's result and compute metrics
                gbest_values = np.load(data_path) * -1
                mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
                error_val = abs(mean_val[-1] - function_opt_val)
                row_data[(optimizer, 'mean')] = mean_val[-1]
                row_data[(optimizer, 'lower')] = lower_val[-1]
                row_data[(optimizer, 'upper')] = upper_val[-1]
                row_data[(optimizer, 'optimum')] = function_opt_val
                row_data[(optimizer, 'error')] = error_val
                
                # Additional logging or plotting can be added here as needed
                
            except Exception as e:
                print(f"Error processing {optimizer} for function {function_id}: {e}")
                # Fill missing values if any error occurs
                for metric in ["mean", "lower", "upper", "optimum", "error"]:
                    row_data[(optimizer, metric)] = np.nan
        
        # After collecting data for all optimizers, add the row to the DataFrame
        #df = df.append(pd.Series(row_data, name=function_id))
        df.loc[function_id] = row_data
    
    # Save the DataFrame to CSV
    df.to_csv(f"{base_path}/results.csv")
    
    # Log the DataFrame if Neptune logger is provided
    if neptune_logger:
        neptune_logger["test/results"].upload(f"{base_path}/results.csv")
    return df

In [ ]:
other_algo_path = "other_algorithms/2024-03-10_10-38-15"
model_base_path = "../new_test_results/2024-03-10_12-23-13"

columns = pd.MultiIndex.from_product([model_lists, 
                                          ["mean", "lower", "upper", "optimum", "error"]],
                                         names=['optimizer', 'metric'])
# Initialize an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)
    
function_ids = ["f01", "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
                "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29", "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "f38", "f39", "f40",
                "f41", "f42", "f43", "f44", "f45", "f46", "f47", "f48", "f49", "f50"]

other_algos = ["DE", "GA", "PSO", "SA"]
deephive_algos = ["model102", "model103", "model104"]
model_lists = other_algos + deephive_algos

for function_id in function_ids:
    row_data = {}
    function_info = function_selector.get_function(function_id)
    function_opt_val = function_info["global_min"]
    for model in model_lists:
        if model in other_algos:
            model_path = f"{other_algo_path}/{function_id}/{model}_histories.npy"
        else:
            model_path = f"{model_base_path}/{model}/deephive/{function_id}/{function_id}_gbest_history.npy"
            
        print(model_path)
        try:
            # Attempt to load the optimizer's result and compute metrics
            gbest_values = np.load(model_path) * -1 if model in deephive_algos else np.load(model_path)
            mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
            error_val = abs(mean_val[-1] - function_opt_val)
            row_data[(model, 'mean')] = mean_val[-1]
            row_data[(model, 'lower')] = lower_val[-1]
            row_data[(model, 'upper')] = upper_val[-1]
            row_data[(model, 'optimum')] = function_opt_val
            row_data[(model, 'error')] = error_val
            
            # Additional logging or plotting can be added here as needed
        except Exception as e:
            print(f"Error processing {model} for function {function_id}: {e}")
            # Fill missing values if any error occurs
            for metric in ["mean", "lower", "upper", "optimum", "error"]:
                row_data[(model, metric)] = np.nan
                
        df.loc[function_id] = row_data
        
df.to_excel("results.xlsx")

In [ ]:
other_algo_path = "other_algorithms/2024-03-10_10-38-15"
model_base_path = "../new_test_results/2024-03-10_12-23-13"
save_dir = "optimization_results/"

columns = pd.MultiIndex.from_product([model_lists, 
                                          ["mean", "lower", "upper", "optimum", "error"]],
                                         names=['optimizer', 'metric'])
# Initialize an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)
    
function_ids = ["f01", "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
                "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29", "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "f38", "f39", "f40",
                "f41", "f42", "f43", "f44", "f45", "f46", "f47", "f48", "f49", "f50"]

other_algos = ["DE", "GA", "PSO", "SA"]
deephive_algos = ["model102", "model103", "model104"]
model_lists = other_algos + deephive_algos

for function_id in function_ids:
    row_data = {}
    function_info = function_selector.get_function(function_id)
    function_opt_val = function_info["global_min"]
    for model in model_lists:
        if model in other_algos:
            model_path = f"{other_algo_path}/{function_id}/{model}_histories.npy"
        else:
            model_path = f"{model_base_path}/{model}/deephive/{function_id}/{function_id}_gbest_history.npy"
            
        print(model_path)
        try:
            # Attempt to load the optimizer's result and compute metrics
            gbest_values = np.load(model_path) * -1 if model in deephive_algos else np.load(model_path)
            mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
            error_val = abs(mean_val[-1] - function_opt_val)
            row_data[(model, 'mean')] = mean_val[-1]
            row_data[(model, 'lower')] = lower_val[-1]
            row_data[(model, 'upper')] = upper_val[-1]
            row_data[(model, 'optimum')] = function_opt_val
            row_data[(model, 'error')] = error_val
            
            # Additional logging or plotting can be added here as needed
        except Exception as e:
            print(f"Error processing {model} for function {function_id}: {e}")
            # Fill missing values if any error occurs
            for metric in ["mean", "lower", "upper", "optimum", "error"]:
                row_data[(model, metric)] = np.nan
                
        df.loc[function_id] = row_data
        
df.to_excel(f"{save_dir}/results.xlsx")

In [ ]:
import pandas as pd
import numpy as np
import os

# Define your paths, lists, and function IDs as before...
other_algo_path = "other_algorithms/2024-03-10_10-38-15"
model_base_path = "../new_test_results/2024-03-10_12-23-13"
save_dir = "optimization_results/"
other_algos = ["DE", "GA", "PSO", "SA"]
deephive_algos = ["model102", "model103", "model104"]
model_lists = other_algos + deephive_algos
function_ids = [f"f0{i}" if i < 10 else f"f{i}" for i in range(1, 51)]

for function_id in function_ids[:]:
    # Temporarily load one file to determine the maximum number of iterations
    sample_model_path = os.path.join(other_algo_path, f"{function_ids[0]}/{other_algos[0]}_histories.npy")
    sample_data = np.load(sample_model_path)
    max_iterations = 1010  # Default value for the sample data

    # Create column names for each iteration and metric
    columns = []
    for model in model_lists:
        for metric in ["mean", "lowest"]:
            columns.extend([f"{model}_{metric}"])

    # Initialize the DataFrame for this function with dynamic columns
    df = pd.DataFrame(columns=columns)  # Single row to populate

    for model in model_lists:
        try:
            if model in other_algos:
                model_path = os.path.join(other_algo_path, f"{function_id}/{model}_histories.npy")
            else:
                model_path = os.path.join(model_base_path, f"{model}/deephive/{function_id}/{function_id}_gbest_history.npy")

            gbest_values = np.load(model_path)
            if model in deephive_algos:
                gbest_values *= -1  
            
            if gbest_values.shape[1] > max_iterations:
                max_iterations = gbest_values.shape[1]
                
            # Calculate the mean and lowest values for each iteration
            mean_vals, lowest_vals, _ = mean_confidence_interval(gbest_values)
            # Pad the arrays to the maximum number of iterations
            mean_vals = np.pad(mean_vals, (0, max_iterations - len(mean_vals)), constant_values=np.nan)
            lowest_vals = np.pad(lowest_vals, (0, max_iterations - len(lowest_vals)), constant_values=np.nan)
            
            df[f"{model}_mean"] = mean_vals
            df[f"{model}_lowest"] = lowest_vals
            
        except Exception as e:
            #import traceback; traceback.print_exc();
            print(f"Error processing {model} for function {function_id}: {e}")

    # Ensure the save directory exists and save the DataFrame as a CSV
    os.makedirs(save_dir, exist_ok=True)
    df.to_excel(os.path.join(save_dir, f"{function_id}_results.xlsx"), index=False)


In [52]:
model = "SA"
function_id = "f01"
model_path = os.path.join(other_algo_path, f"{function_id}/{model}_histories.npy")

In [53]:
gbest_values = np.load(model_path)

In [56]:
gbest_values.shape

(10, 58)

In [54]:
mea, low, upp = mean_confidence_interval(gbest_values)

In [55]:
mea

array([11.50531703, 11.50531703, 11.50531703, 11.50531703, 11.50531703,
       11.50531703, 11.50531703, 11.50531703, 11.50531703, 11.50531703,
       11.50531703, 11.50531703, 11.50531703, 11.50531703, 11.50531703,
       11.50531703, 11.50531703, 11.50531703, 11.50531703, 11.50531703,
       11.50531703, 11.50531703, 11.50531703, 11.50531703, 11.3209817 ,
       11.3209817 , 11.28079852, 11.28079852, 11.28079852, 11.28079852,
       11.28079852, 11.207999  , 11.08512319, 10.85370496, 10.81490854,
       10.58790761, 10.42064126, 10.35445952, 10.18244036, 10.09318225,
        9.88690437,  9.26643107,  8.8196819 ,  8.27402997,  7.68276629,
        7.26937584,  7.00477172,  6.6451879 ,  6.18398699,  5.78734571,
        5.18046689,  4.62893227,  4.43066991,  4.15174814,  3.65021795,
        3.42069952,  3.07788575,  2.90676415])